In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm

from feature_extractor import FeatureExtractor

In [2]:
# load old feature_extraction file:
anno_features = pickle.load(open('../../pvc-meteor/features/extracted_features_06-01-2023-18-39.pkl', 'rb'))

In [3]:
extractor = FeatureExtractor(15)
out_dict = dict()

for name in tqdm(anno_features['annotations'].keys()):
    anno_dict = extractor.annotation_processor(name)
    out_dict.update({name: anno_dict})

with open('../../pvc-meteor/features/annotations.pickle', 'wb') as file:
    pickle.dump(out_dict, file)

100%|██████████| 1213/1213 [42:05<00:00,  2.08s/it] 


## get label distribution

In [9]:
np.sum(out_dict['REC_2020_10_12_03_08_38_F.MP4']['anno'], axis=0)

array([  0.,   0., 104.,   0.,   0.,   0., 104.,   0.])

In [13]:
out_dict['REC_2020_10_12_03_08_38_F.MP4']['feature_length']

104

In [30]:
total_frames = 0
cat_array = np.zeros((8,))

for name in tqdm(out_dict.keys()):
    total_frames += out_dict[name]['feature_length']
    new_cat = np.sum(out_dict[name]['anno'], axis=0)
    if new_cat[3] > 0:
        print(name)
        print(np.nonzero(out_dict[name]['anno'][3,:]))

    cat_array = cat_array + new_cat


100%|██████████| 1213/1213 [00:00<00:00, 24844.91it/s]

REC_2020_10_12_17_30_07_F.MP4
(array([2, 6]),)


In [26]:
cat_list = cat_array.tolist()
cat_list[2] - total_frames

-900.0

In [1]:
cat_list

NameError: name 'cat_list' is not defined